# Usage

In [ ]:
from importlib import reload
import os

from obspy.clients.filesystem.sds import Client
from obspy.clients.fdsn import RoutingClient
from obspy.core import UTCDateTime as UTC
from obspy.signal import util

import matplotlib.pyplot as plt
plt.style.use('tableau-colorblind10')

from data_quality_control import processing

## Define parameters

In [ ]:
network = 'GR'
station = 'BFO'
location = ''
channel = 'HHZ'

overlap = 60 #3600
fmin, fmax = (4, 14)
nperseg = 2048
winlen_in_s = 3600
proclen = 24*3600

outdir = '../sample_output/show_processing/'

sds_root = os.path.abspath('../sample_sds/')
inventory_routing_type = "eida-routing"

You can use the Clients directly to check the content
of the database. Note though, that this can take some time
if your database is large.

In [ ]:
sdsclient = Client(sds_root)
invclient = RoutingClient(inventory_routing_type)

sdsclient.get_all_nslc()

## Process raw data

Use the `processing` module to extract amplitudes and
power spectral densities (PSD) from the raw seismic data.

In [ ]:
startdate = UTC("2020-12-20")
enddate = UTC("2021-01-15")

In [ ]:
reload(processing)
proc_params = processing.ProcessingParameters(overlap, 
                                (fmin, fmax), nperseg,
                                winlen_in_s, proclen)
sdsp = processing.SDSDataBaseProcessor(network, station, channel, 
                                    sds_root, inventory_routing_type, 
                                        outdir, proc_params)

In [ ]:
sdsp.process(startdate, enddate, outdir="../tmp_data/")

In [ ]:
reload(processing)
processor = processing.RawDataProcessor(
        network=network, 
        station=station,
        channel=channel,
        sds_root=sds_root,
        inventory_routing_type=inventory_routing_type,)

In [ ]:
%%time
#it -n1 -r7
processor.process(startdate, enddate, outdir,
                        overlap=overlap, 
            winlen_in_s=winlen_in_s,
        nperseg=nperseg, proclen=proclen,
                 amplitude_frequencies=(fmin, fmax))

## View processed data

With the `analysis` module, the processed data can be accessed 
and visualized.

In [ ]:
from data_quality_control import analysis
import numpy as np

In [ ]:
from IPython.core.display import display, HTML

In [ ]:
startdate = UTC("2020-12-28")
enddate = UTC("2021-01-15")

In [ ]:
stationcode = "{}.{}.{}.{}".format(network, station, 
                                   location, channel)

In [ ]:
reload(analysis)
analyzer = analysis.Analyzer(startdate, enddate, outdir, stationcode)

In [ ]:
adata = analyzer.get_data(["amplitudes", "psds"], 
                         stime="00:00", etime="23:00"
                        #stime="19:00", etime="05:00"
                         )

In [ ]:
fig = analyzer.plot_amplitudes()

In a notebook or script you could simply run:

```python
fig.show()
```

In [ ]:
display(HTML(fig.to_html()))

In [ ]:
fig_psd = analyzer.plot_psds(np.log)

In [ ]:
display(HTML(fig_psd.to_html()))